In [ ]:
inp = 0
#Using thread to store the user input for task classification
def background():
    global inp
    while True:
        inp = int(input())

def collectData():
    return 0

#Now threading1 runs regardless of user input
threading1 = threading.Thread(target=background)
threading1.daemon = True
threading1.start()

ser0 = serial.Serial('/dev/ttyACM0', 115200) # Establish the connection on a specific port
ser1 = serial.Serial('/dev/ttyACM1', 115200) # Establish the connection on a specific port
counter0 = 0 
counter1 = 0 
fhandle  = open("imu.txt", 'ab')
fhandle0 = open("imu0.txt", 'ab')
fhandle1 = open("imu1.txt", 'ab')
ihandle0 = open("intent0.txt", 'ab')
ihandle1 = open("intent1.txt", 'ab')
imu0    = []  #contains tstamps with imu0 data
imu1    = []  #contains tstamps with imu1 data
imu     = []

intent0 = []
intent1 = []
ser0.flush()
ser1.flush()
start0 = time.time()
start1 = time.time()
t0     = time.time()
t1     = time.time()

#FFT might have the User Activity information as well i.e. running, walking, sitting etc. which can also be used for counterchecks for intent classification
#Or the above info can be captured as well in the SVM classifier
while True:
    try:
        counter0 +=1
        counter1 +=1
        #ser.write(str(chr(counter))) # Convert the decimal number to ASCII then send it to the Arduino
        data0 = ser0.readline().rstrip()
        data0 = data0.split()
        data1 = ser1.readline().rstrip()
        data1 = data1.split()
        if ((len(data0) == 6) & (len(data1) == 6) & (inp<2)):
            imu0.append([float(time.time()), int(data0[0]), int(data0[1]), int(data0[2]), int(data0[3]), int(data0[4]), int(data0[5])])
            imu1.append([float(time.time()), int(data1[0]), int(data1[1]), int(data1[2]), int(data1[3]), int(data1[4]), int(data1[5])])
            imu.append([float(time.time()),  (int(data0[0])*2.0)/ 32768.0, (int(data0[1])*2.0)/ 32768.0, (int(data0[2])*2.0)/ 32768.0, (int(data0[3])*250.0)/32768.0, (int(data0[4])*250.0)/32768.0, (int(data0[5])*250.0)/32768.0, (int(data1[0])*2.0)/ 32768.0, (int(data1[1])*2.0)/ 32768.0, (int(data1[2])*2.0)/ 32768.0, (int(data1[3])*250.0)/32768.0, (int(data1[4])*250.0)/32768.0, (int(data1[5])*250.0)/32768.0])
            intent0.append(inp)
            intent1.append(inp)
        if counter0 == 255:
            print("frequency:", 255/(time.time()-t0))
            t0 = time.time()
            counter0 = 0
        if counter1 == 255:
            print("frequency:", 255/(time.time()-t1))
            t1 = time.time()
            counter1 = 0
            
    except KeyboardInterrupt:
        end=time.time()
        d_time0=end-start0
        d_time1=end-start1
        np.savetxt(fhandle0, np.array(imu0),delimiter=",")
        fhandle0.close()
        np.savetxt(fhandle1, np.array(imu1),delimiter=",")
        fhandle1.close()
        np.savetxt(ihandle0, np.array(intent0),delimiter=",")
        ihandle0.close()
        np.savetxt(ihandle1, np.array(intent1),delimiter=",")
        ihandle1.close()
        np.savetxt(fhandle, np.array(imu),delimiter=",")
        fhandle.close()
        print("Avg Frequency is :: ", len(imu0)/d_time0)
        print("Avg Frequency is :: ", len(imu1)/d_time1)
        ser0.close()
        ser1.close()
        pass

In [ ]:
#ser0 = serial.Serial('/dev/ttyACM0', 115200) # Establish the connection on a specific port 0
#ser1 = serial.Serial('/dev/ttyACM1', 115200) # Establish the connection on a specific port 1
#ser0.flush()
#ser1.flush()

intent_thres_imu = 10
imu_data   = []
pred_count = 0

clf = joblib.load('weights/1s_6sps.pkl')
print(clf)
    
print("Intention Probably is to pick up the bottle")
data_count = 0
while(True):
    try:
        while(data_count<30):       #read data from imu
            data0 = ser0.readline().rstrip()
            data0 = data0.split()
            data1 = ser1.readline().rstrip()
            data1 = data1.split()
            if (len(data0) == 6 & len(data1) == 6):
                imu_data.append([float(time.time()),  int(data0[0]), int(data0[1]), int(data0[2]), int(data0[3]), int(data0[4]), int(data0[5]), int(data1[0]), int(data1[1]), int(data1[2]), int(data1[3]), int(data1[4]), int(data1[5])])
                data_count = data_count + 1
                
        data_count = 0
        bucket    = get_buckets(np.array(imu_data[-30:]), 1, 30, 40)   #send the last 30 elements data to get_samples function
        predicted = clf.predict(np.reshape(bucket, (1, 40)))           #reshaping and sending for classification
        print(bucket)
        if(predicted == 1):
            pred_count = pred_count + 1
            print("IMU intent Prediction Count", pred_count)
            if(pred_count>intent_thres_imu):
                print("Intended Action is XYZ")
            else:
                print("IMU intent Prediction Count", pred_count)
                pred_count = pred_count - 1
    except KeyboardInterrupt:
        ser0.flush()
        ser0.close()
        ser1.flush()
        ser1.close()
        pass
        #keep a counter for it as well
        #wait for threshold to predict the intection

In [ ]:
thread = threading.Thread(target=getRefInput)
thread.daemon = True
thread.start()

inp = '3'

file, flist = createFile('data/11testfile.h5')

imu_num = 2
imu_data_chunk = 100
imu_data  = deque([deque([]) for _ in range(imu_num)])
tstamp    = deque([deque([]) for _ in range(imu_num)])

#Open serial port for reading data from Intel Curie IMU
ser0 = serial.Serial('/dev/ttyACM0', 115200) # Establish the connection on a specific port 0
ser1 = serial.Serial('/dev/ttyACM1', 115200) # Establish the connection on a specific port 1
ser0.flush()
ser1.flush()
 
while inp!='s':  #as of now continous loop but should be more adaptive and intuitive #take care of 'garbage data'
    data0 = ser0.readline().rstrip()
    data0 = data0.split()
    data1 = ser1.readline().rstrip()
    data1 = data1.split()
    if (len(data0) == 6 & len(data1) == 6):
        tstamp[connection].append(time.time()) 
        imu_data.append([int(data0[0]), int(data0[1]), int(data0[2]), int(data0[3]), int(data0[4]), int(data0[5]), 0, 0, 0, int(data1[0]), int(data1[1]), int(data1[2]), int(data1[3]), int(data1[4]), int(data1[5]), 0, 0, 0])
        #imu_data[connection].append(value_data[2::2])     #storing all 9-axis imu data

        if((len(imu_data[connection]) == imu_data_chunk) or (inp == 's')):
            saveData(flist, connection, imu_data, tstamp)
            print("Average Sampling Freq. of", imu_data_chunk, "elements is ::", getFreq(tstamp, connection, show=False))     #imu_data_chunk/(tstamp[connection][0]-tstamp[connection][299])
            imu_data[connection].clear()
            tstamp[connection].clear()

In [ ]:
def get_frequencies(data, timestamps):
    num_seconds = float(timestamps[-2] - timestamps[0])
    samples_per_second = len(data) / num_seconds
    num_samples = len(data)
    oscilations_per_sample = [float(oscilations) / num_samples for oscilations in range(0, num_samples)]
    return [ops * samples_per_second for ops in oscilations_per_sample]

def get_buckets(data, first, last, num_buckets, hertz_cutoff=float(5)):
    # Transform all of the original data to be a single component along the first principal component
    pca = PCA(n_components=1, copy=True, whiten=True)
    transformed_dataset = PCA.fit_transform(pca, data)
    #print(pca.explained_variance_ratio_)
    slice=transformed_dataset[first:last]

    transformed = fft.fft(slice)
    absolute = [abs(complex) for complex in transformed]

    frequencies = get_frequencies(data, (data.T)[0])

    buckets = [0 for i in range(num_buckets)]
    width = hertz_cutoff / num_buckets
    sum_of_buckets = 0.0000001
    for i in range(1, len(absolute)):
        index = int(frequencies[i] / width)
        if index >= num_buckets:
            break;
        buckets[index] += absolute[i]
        sum_of_buckets += absolute[i]

    if 0: #arguments['--normalize']:
        buckets = map(lambda x: x/sum_of_buckets, buckets)

    return buckets

def get_samples(data, target):
    result = []
    intent = []
    segmentsize=30
    stride=5            # Reduce this to very little to get very large trainingsets
    noOfBuckets=40
    clas = np.array(target)
    for  start in range(0, len(data) - segmentsize, stride):
        if ((sum(clas[start: start + stride])<stride) & (sum(clas[start: start + stride])>0)): #((clas[start: start + stride]==).all()):
            continue
        if start + segmentsize <= len(data):
            segments_buckets = get_buckets(data, start, start + segmentsize, noOfBuckets)
            #print(segments_buckets)
            intent.append(clas[start])
            result.append(segments_buckets)
    return result, intent

def getPCASample(data, target, isPCA=True):
    result = []
    intent = []
    segmentsize=30
    stride=2            # Reduce this to very little to get very large trainingsets
    noOfBuckets=40
    clas = np.array(target)
    pca = PCA(n_components=1, copy=True, whiten=True)
    
    #Pre-processing the data
    arr = np.array(data).T
    new_arr = arr[1:13]
    new_arr[0]  = (new_arr[0]*2)/ 32768.0
    new_arr[1]  = (new_arr[1]*2)/ 32768.0
    new_arr[2]  = (new_arr[2]*2)/ 32768.0
    new_arr[3]  = (new_arr[3]*250)/ 32768.0
    new_arr[4]  = (new_arr[4]*250)/ 32768.0
    new_arr[5]  = (new_arr[5]*250)/ 32768.0
    new_arr[6]  = (new_arr[6]*2)/ 32768.0
    new_arr[7]  = (new_arr[7]*2)/ 32768.0
    new_arr[8]  = (new_arr[8]*2)/ 32768.0
    new_arr[9]  = (new_arr[9]*250)/ 32768.0
    new_arr[10] = (new_arr[10]*250)/ 32768.0
    new_arr[11] = (new_arr[11]*250)/ 32768.0
    
    new_arr = new_arr.T
    
    if isPCA:
        transformed_dataset = PCA.fit_transform(pca, new_arr)
    else:
        transformed_dataset = new_arr
    #transformed_dataset = PCA.fit_transform(pca, data)
    
    for  start in range(0, len(data) - segmentsize, stride):
        if ((sum(clas[start: start + stride])<stride) & (sum(clas[start: start + stride])>0)): #((clas[start: start + stride]==).all()):
            continue
        if start + segmentsize <= len(data):
            intent.append(clas[start])
            result.append(np.reshape(transformed_dataset[start:start+segmentsize], (30,)))
    return result, intent

def plotDataset(start, end):
    imu_data = loadData("data/imu.txt").T
    intent   = loadData("data/intent0.txt")
    abs_data = np.abs(imu_data)
    for i in range(6):
        data = abs_data[i+1]
        pdata= data[start:end]
        plt.plot((pdata-min(pdata))/(max(pdata)-min(pdata)))
        plt.plot(intent[start:end])
        plt.show()
        
    return imu_data, intent